## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

In [2]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

1. Общее количество строк в таблице: 21525
2. Названия всех столбцов корректные и не требуют переименования
3. В столбцах 'days_employed' и 'total_income' есть пропущенные значения. Количество пропущенных значений в каждом столбце: 2174 (10% от общего количества)

### Обработка пропусков

In [4]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце 'children' есть значения '-1' и '20'. Видимо, это ошибки ввода. 
Заменяю на 1 и 2, это оптимально так как ошибочных данных менее 1%.

In [5]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [6]:
#Проверяем результат
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [7]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Столбец 'gender' - есть одно пропущенное значение 'XNA'. Заменяю на F случайным образом, так как это не повлияет на результат исследования.

In [8]:
df['gender'] = df['gender'].replace('XNA', 'F')

In [9]:
#Проверяем результат
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [10]:
unique_dob_years = df['dob_years'].unique()
unique_dob_years.sort()
print(unique_dob_years)

[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]


In [11]:
df['dob_years'].value_counts(normalize=True, bins = 5)

(30.0, 45.0]      0.395075
(45.0, 60.0]      0.327851
(15.0, 30.0]      0.172962
(60.0, 75.0]      0.099419
(-0.076, 15.0]    0.004692
Name: dob_years, dtype: float64

В столбце 'dob_years' есть значения '0' (менее 0,5%). Заменяю медианой по всему столбцу. 

In [12]:
median_dob_years = df['dob_years'].median()
def change_dob_years(years):
    if years == 0:
        return median_dob_years
    return years

df['dob_years'] = df['dob_years'].apply(change_dob_years)

In [13]:
#Проверяем результат
unique_dob_years = df['dob_years'].unique()
unique_dob_years.sort()
print(unique_dob_years)

[19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71. 72.
 73. 74. 75.]


In [14]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [15]:
print(df['days_employed'].loc[df['days_employed'] >= 0 ].describe())

count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64


In [16]:
print(df['days_employed'].loc[df['days_employed'] < 0 ].describe())

count    15906.000000
mean     -2353.015932
std       2304.243851
min     -18388.949901
25%      -3157.480084
50%      -1630.019381
75%       -756.371964
max        -24.141633
Name: days_employed, dtype: float64


В столбце 'days_employed' присутствуют значения с минусом, а положительные значения нереалистично большие. Видимо, данные были собраны с использованием разных методов. 
Установим максимальное значение трудового стажа - 20 075 дней (55 лет трудового стажа умноженные на 365).
Как видно, значения с минусом похожи на правду, их приведем к модулю. 

In [17]:
df['days_employed'] = df['days_employed'].abs()

Предположим, что нереалистично большие значения трудового стажа это значения в часах. В таком случае переведем часы в дни.

In [18]:
max_days_employed = 55*365
def converting_hours_to_days(days):
    if days > max_days_employed:
        return days/24
    return days

df['days_employed'] = df['days_employed'].apply(converting_hours_to_days)

In [19]:
#Проверяем результат
df['days_employed'].describe()

count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

В целом данные выглядят реалистично.

In [20]:
df.groupby('dob_years')['days_employed'].agg(['mean','median','max'])

,mean,median,max
dob_years,,,
19.0,633.678086,724.492610,1020.183130
20.0,684.944308,674.838979,1547.804060
21.0,709.440930,618.733817,1948.929810
22.0,860.696860,703.310078,13948.510826
23.0,827.309437,690.204208,2571.577635
24.0,1026.405485,947.731043,3385.118897
25.0,1088.406453,919.199388,3896.705122
26.0,1278.050735,1083.658132,16224.881982
27.0,1448.262099,1166.212160,15812.170938


При дополнительной проверке видно, что максимальные значения стажа во многих случаях нереалистичны по отношению к возрасту клиентов. Также странным выглядит скачок значения медианы после 56 лет. 
Оставляем полученные значения, так как для решения этой проблемы нужна дополнительная информация. На итоговые выводы признак 'days_employed' не повлияет. 

Также мы знаем, что в данном столбце есть пропущенные значения. Можем ничего не делать с пропущенными значениями, так как у человека просто может не быть трудового стажа.  Но если предположить, что кредит одобряют только при наличии минимального трудового стажа, то пропущенные значения  - это ошибки при сборе данных. Заполняем пропуски средней по возрасту.

In [21]:
df['days_employed'] = df.groupby('dob_years') ['days_employed'].transform(lambda x: x.fillna(x.mean()))

In [22]:
#Проверяем результат
df['days_employed'].count()

21525

In [23]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

В столбце 'total_income' есть пропущенные значения. Можем ничего не делать с пропущенными значениями, так как у человека может не быть дохода.  Но если предположить, что кредит не одобряют без дохода, то пропущенные значения  - это ошибки при сборе данных. Заполняем пропуски медианой по типу занятости.

In [24]:
df['total_income'] = df.groupby('income_type') ['total_income'].transform(lambda x: x.fillna(x.median()))

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


### Вывод

1. В столбцах days_employed и total_income были найдены и заменены пропущенные значения. 
2. В столбцах children, dob_years, gender были найдены и заменены ошибочные значения.
3. Причинами могут быть ошибки ввода данных, сокрытие информации, фрод.

### Замена типа данных

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [27]:
df['days_employed'] = df['days_employed'].astype('int')

In [28]:
df['dob_years'] = df['dob_years'].astype('int')

In [29]:
df['total_income'] = df['total_income'].astype('int')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

1. Столбцы, в которых необходимо изменить тип данных float на int:
days_employed,
dob_years,
total_income.

2. Применила метод astype(), так как этот метод обеспечит перевод в необходимый нам тип данных int.

### Обработка дубликатов

In [31]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [32]:
df['education'] = df['education'].str.lower()

In [33]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [34]:
df.duplicated().sum()

72

In [35]:
df = df.drop_duplicates().reset_index(drop = True)

In [36]:
#Проверяем результат
df.duplicated().sum()

0

In [37]:
df['family_status'] = df['family_status'].str.lower()

### Вывод

1. Для корректного удаления дубликатов значения столбца 'education' приведены к одинаковому регистру.
2. После удаляем дубликаты.

### Лемматизация

In [38]:
from pymystem3 import Mystem
m = Mystem()

Лемматизируем столбец 'purpose':

In [39]:
def lemmas_search(column):
    return m.lemmatize(column)

df['purpose_lemmas'] = df['purpose'].apply(lemmas_search)

In [40]:
lemmas = []
for row in df['purpose_lemmas']:
    for lemma in row:
        lemmas.append(lemma)

In [41]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 33568, '\n': 21453, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2323, 'свой': 2230, 'на': 2221, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 767, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Проанализировав леммы, можно выделить категории:
1. Недвижимость
2. Автомобиль
3. Образование
4. Свадьба

Присваиваем категорию, создав новый столбец:

In [42]:
categories = set(['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба'])

def categorisation(data):
    n = set(data['purpose_lemmas']) & categories
    if len(n) > 0:
        for i in n:
            return i
    return 'нет категории'

df['purpose_category'] = df.apply(categorisation, axis = 1)

In [43]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


Объединяем категорию 'жилье' и 'недвижимость' в одну общую категорию 'недвижимость':

In [44]:
df['purpose_category'] = df['purpose_category'].replace('жилье', 'недвижимость')

In [45]:
#Проверяем результат
df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_category, dtype: int64

### Вывод

После анализа лемм вручную были выделены 4 категории целей. Я объединила 'жилье' и 'недвижимость' в одну категорию, без деления на жилую и коммерческую недвижимость. В итоге все данные полностью поделились на 4 категории (Total: 21 453):
1. Недвижимость
2. Автомобиль
3. Образование
4. Свадьба

### Категоризация данных

In [46]:
def children_group(children):
    if children == 0:
        return 'детей нет'
    if 1 <= children <= 2:
        return '1-2 ребенка'
    return 'многодетные'

df['children_income_category'] = df['children'].apply(children_group)

In [47]:
df['children_income_category'].value_counts()

детей нет      14090
1-2 ребенка     6983
многодетные      380
Name: children_income_category, dtype: int64

In [48]:
def total_income_group(income):
    if income < 100000:
        return 'до 100 000'
    if 100000 <= income < 300000:
        return 'от 100 000 до 300 000'
    return 'более 300 000'

df['total_income_category'] = df['total_income'].apply(total_income_group)

In [49]:
df['total_income_category'].value_counts()

от 100 000 до 300 000    15507
до 100 000                4463
более 300 000             1483
Name: total_income_category, dtype: int64

### Вывод

Для того, чтобы ответить на вопроcы задания, были категоризированы следующие признаки:
1. Наличие детей
2. Доходы

### Шаг 3. Ответьте на вопросы

**1. Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [50]:
import numpy as np

In [51]:
children_pivot = df.pivot_table(index=['children_income_category'], values=['debt'])
children_pivot_sort = children_pivot.reindex(children_pivot['debt'].sort_values(ascending=False).index)

In [52]:
children_pivot_sort

,debt
children_income_category,
1-2 ребенка,0.092654
многодетные,0.081579
детей нет,0.075444


### Вывод

Процент клиентов с задолженностью в группе:
1. 1-2 ребенка	 - 9.3%
2. многодетные	 - 8%
3. детей нет	 - 7.5%

Самый большой процент просрочки среди семей с 1-2 детьми. Самый маленький - у бездетных заемщиков.
У семей с детьми процент просрочки меньше среди многодетных семей. Возможно, это связано со льготами, которые предоставляются многодетным семьям.

Можно сделать вывод, что наличие детей повышает вероятность просрочки платежа.

**2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [53]:
family_status_pivot = df.pivot_table(index=['family_status'], values=['debt'])
family_status_sort = family_status_pivot.reindex(family_status_pivot['debt'].sort_values(ascending=False).index)

In [54]:
family_status_sort

,debt
family_status,
не женат / не замужем,0.097509
гражданский брак,0.093494
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


### Вывод

Процент клиентов с задолженностью в группе:
1. не женат / не замужем - 9.8%
2. гражданский брак	- 9.3%
3. женат / замужем	- 7.5%
4. в разводе - 7.1%
5. вдовец / вдова - 6.5%

Самый большой процент просрочки у тех, кто одинок или чей брак не зарегестрирован официально. Среди людей, которые находятся или были в браке, процент просрочки меньше.

Можно сделать вывод, что семейное пложение заемщиков влияет на вероятность просрочки по кредиту.

**3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [55]:
total_income_pivot = df.pivot_table(index=['total_income_category'], values=['debt'])
total_income_sort = total_income_pivot.reindex(total_income_pivot['debt'].sort_values(ascending=False).index)

In [56]:
total_income_sort

,debt
total_income_category,
от 100 000 до 300 000,0.082608
до 100 000,0.079319
более 300 000,0.071477


### Вывод

Процент клиентов с задолженностью в группе:
1. от 100 000 до 300 000 - 8.3%
2. до 100 000	- 7.9%
3. более 300 000	- 7.1%

Наибольший процент просрочек у клиентов с доходом в диапазоне от 100 т. до 300 т., чуть меньше у клиентов с доходом до 100 т.
Наименьший процент просрочек у самой платежеспособной группы клиентов с доходом более 300 т.

Не было выявлено линейной взаимосвязи между уровнем дохода и процентом просрочки. Самый большой процент просрочки у самой многочисленной группы со средним доходом. 

**4. Как разные цели кредита влияют на его возврат в срок?**

In [57]:
purpose_pivot = df.pivot_table(index=['purpose_category'], values=['debt'])
purpose_sort = purpose_pivot.reindex(purpose_pivot['debt'].sort_values(ascending=False).index)

In [58]:
purpose_sort

,debt
purpose_category,
автомобиль,0.093590
образование,0.092200
свадьба,0.080069
недвижимость,0.072334


### Вывод

Процент клиентов с задолженностью в группе:
1. автомобиль - 9.3%
2. образование	- 9.2%
3. свадьба	- 8%
4. недвижимость - 7.2% 

Самый высокий процент просрочек среди клиентов, взявших кредит на автомобиль и образование. Значительно меньший процент просрочки по кредитам на недвижимость. 

Можно сделать вывод, что клиенты, которые берут кредит для операций с недвижимостью, надежнее остальных.

### Шаг 4. Общий вывод

Цель исследования  - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

На этапе предобработки данных были выявлены ошибочные и пропущенные значения. Пропущенные значения составили 11% от общего числа и были заменены средним или медианой, в зависимости от признака.

Общий процент просрочки: 8.8%

Общие выводы:
1. Наличие детей увеличивает вероятность просрочки по кредиту.
2. Отсутствие опыта семейной жизни увеличивает вероятность просрочки по кредиту.
3. Увеличение дохода не напрямую уменьшает вероятность просрочки.
4. По кредитам на недвижимость наименьший процент просрочек.